# 第11章 ファンクターからアプリカティブファンクターへ

### 帰ってきたファンクター

ファンクターは文脈を持った値とみなすことができる  
Maybe値は「計算が失敗したかもしれない」という文脈を、リストは「複数の値を同時に取るかもしれない」という文脈を持つ  

fmapはこういった文脈を保ったまま関数を値に適用する  

### ファンクターとしてのI/Oアクション

IOもファンクターの一種  

```
instance Functor IO where
    fmap f action = do
        result <- action
        return (f result)
```



In [2]:
main = do
    line <- fmap reverse getLine
    putStrLn $ "You said " ++ line ++ " backwards!"
    putStrLn $ "Yes, you really said " ++ line ++ " backwards!"
    

In [3]:
main

You said egohegohegoh backwards!
Yes, you really said egohegohegoh backwards!

In [5]:
-- ファンクターの中身を１つではなく複数関数使いたいなら関数合成がよさげ

import Data.Char
import Data.List

main = do
    line <- fmap (intersperse '-' . reverse . map toUpper) getLine
    putStrLn line

main

### ファンクターとしての関数

`(->) r` もファンクター  
`r -> a` は `(->) r a` と書き換えられる  

実装  

```
instance Functor ((->) r) where
    fmap f g = (\x -> f (g x))
```

fmapの型  

```
fmap :: (a -> b) -> f a -> f b

fmap :: (a -> b) -> ((->) r a) -> ((->) r b)

fmap :: (a -> b) -> (r -> a) -> (r -> b)
```

aからbへの関数と、rからaへの関数を引数に取り、rからbへの関数を返す  
つまり関数合成  
そう考えるとインスタンス宣言はこう書ける  

```
instance Functor ((->) r) where
    fmap = (.)
```

In [7]:
:m + Control.Monad.Instances

In [8]:
:t fmap (*3) (+100)

fmap (*3) (+100) :: forall b. Num b => b -> b

In [11]:
fmap (* 3 ) (+ 100) 1

303

In [13]:
:t (.)

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

In [14]:
:t fmap

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

fmapは関数を取って「元の関数に似てるけどファンクター値を取って、ファンクター値を返す関数」を返す関数  
関数 a -> bを受け取って、関数 f a -> f bを返す操作のことを 持ち上げ `lifting` という  


In [15]:
:t fmap (*2)

fmap (*2) :: forall (f :: * -> *) b. (Functor f, Num b) => f b -> f b

In [16]:
:t fmap (replicate 3)

fmap (replicate 3) :: forall (f :: * -> *) a. Functor f => f a -> f [a]

fmapについての２通りの考え方  

- fmapは関数とファンクター値を取って、その関数でファンクター値を写して返すもの
- fmapは値から値への関数を取って、それをファンクター値からファンクター値への関数に持ち上げたものを返す関数である


In [17]:
fmap (replicate 3) [1,2,3,4]

[[1,1,1],[2,2,2],[3,3,3],[4,4,4]]

In [18]:
fmap (replicate 3) (Just 4)

Just [4,4,4]

In [19]:
fmap (replicate 3) Nothing

Nothing

次回

## ファンクター則

デュエルスタンバイ